# Preprocess Attributes

Adrian Wiegman | arhwiegman.github.io | adrian.wiegman@usda.gov

2023-06-29

Status: Need to improve representation of continuous variables calculation of summary statistics, e.g. area weighted average.  

---



This notebook produces a dataframe of summarized watershed attributes within various groups subwatersheds 
 
This notebook takes inputs from the following tables located in the project `outputs` directory:


1. __`df_monitoring_point_subs.csv`__ a comma delimited table of subwatershed ids of sampling locations


Variable|Description
---|---
FID | point identification code
Lat | latitude degrees
Lon | longitude degrees
Region_MEP | region > name of MEP report > name of monitoring location
SUBW_NAME | name of subwatershed
PROPORTION | proportion of subwatershed discharge draining to the monitoring location

2. __`df_subs_tt_le5pct_select.pkl`__ a pickled pandas dataframe of subwatershed polygons split by lidar elevation class (5th percentile) and USGS simulated travel time (GT10 or LT10)

Variable|Description
---|---
'OBJECTID'| unique polygon id
'SUBW_NAME'| name of subwatershed in MEP dataset
'ele5pct'| elevation perecentile group within subwatershed GT5% indicates uplands, LT5% indicates the zone seepage that discharge to the terminus are likely to occur 
'Travel_Tim'| travel time group GT10 indicates greater than 10 year travel time for water to reach the terminus, LT10 less than 10 yr travel time
"EMBAY_NAME"| name of the estuarine embayment
"SUBWATER_N"| original subwatershed name includeing travel time distinction

3. __`df_subs_tt_le5_lclu16_select.pkl`__ a pickled pandas dataframe of the identity intersection of `subs_tt_le5pct` with polygons of NRCS SSURGO top 20 soil properties

Variable|Description
---|---
'OBJECTID'| unique polygon id
...|...
'LOC_ID'| tax parcel location id (not available in this table)
"Shape_Length"| polygon perimeter length in meters
"Shape_Area"| polygon area in square meters
"COMPONENT"| name of soil series
"SLOPE"| slope code (0 = water)
"SLOPE_1"| representative slope for slope code (water is null value)
"FRMLNDCLS"| Farmlands classification
'HYDROLGRP'| hydrologic soil group (D is poorly drained or has potential to be poorly drained)
'HYDRCRATNG'| hydric soils rating yes/no
'DRAINCLASS'| Drainage class
'DEP2WATTBL'| Depth to Water Table (cm)
    


4. __`df_subs_tt_le5_soil20_selected.pkl`__ a pickled pandas dataframe of subwatershed polygons split by lidar elevation class (5th percentile) and intersected with 0.5m resolution land cover land use polygons from MassGIS


VARIABLE|DESCRIPTION
---|---
'OBJECTID'| unique polygon id
... | ... 
"COVERCODE"| land cover code NLCD
"COVERNAME"|  land cover name NLCD
"USE_CODE" | specific use code from Massachusetts tax parcel assessor data 
"USEGENCODE"| generalized Use code from Massachusetts tax parcel assessor data 
'USEGENNAME'| generalized use name from Massachusetts tax parcel assessor data 


In [2]:
# iphython options
# delete variables in workspace
%reset -f
#places plots inline
%matplotlib inline
#automatically reloads modules if they are changed
%load_ext autoreload 
%autoreload 2
# this codeblock sets up the environment from jupyter notebooks
setup_notebook = "C:/Users/Adrian.Wiegman/Documents/GitHub/Wiegman_USDA_ARS/MEP/_Setup.ipynb"
%run $setup_notebook # magic command to run the notebook 

***
loading python modules...

  `module_list` contains names of all loaded modules

...module loading complete

***
loading user defined functions...

type `fn_`+TAB to for autocomplete suggestions

 the object `def_list` contains user defined function names:
   fn_get_info
   fn_arcgis_table_to_df
   fn_arcgis_table_to_np_to_pd_df
   fn_try_mkdir
   fn_hello
   fn_recursive_glob_search
   fn_regex_search_replace
   fn_regex_search_0
   fn_arcpy_table_to_excel
   fn_agg_sum_df_on_group
   fn_add_prefix_suffix_to_selected_cols
   fn_calc_pct_cover_within_groups

 use ??{insert fn name} to inspect
 for example running `??fn_get_info` returns:
Signature: fn_get_info(name='fn_get_info')
Source:   
def fn_get_info(name='fn_get_info'):
    '''
    returns the source information about a given function name
    '''
    get_ipython().run_line_magic('pinfo2', '$name')
File:      c:\windows\system32\<ipython-input-1-28f96385f735>
Type:      function

 you can also use `fn_get_info(name={insert f

In [3]:
# read the monitoring points table
filename = "df_monitoring_point_subs.csv"
_ = pd.read_csv(os.path.join(odr,filename))
print(_.info())
display(_.head())
df_points = _
del _ # clear temporary object from memory

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252 entries, 0 to 251
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  252 non-null    int64  
 1   FID         252 non-null    int64  
 2   SUBW_NAME   252 non-null    object 
 3   PROPORTION  252 non-null    float64
 4   Lat         252 non-null    float64
 5   Lon         252 non-null    float64
 6   Region_MEP  252 non-null    object 
dtypes: float64(3), int64(2), object(2)
memory usage: 13.9+ KB
None


,Unnamed: 0,FID,SUBW_NAME,PROPORTION,Lat,Lon,Region_MEP
0,0,1,UpperAcushnetRiver,1.0,41.681859,-70.918844,Buzzards Bay > Acushnet > Acushnet River
1,1,1,NewBedfordReservoir,1.0,41.681859,-70.918844,Buzzards Bay > Acushnet > Acushnet River
2,2,2,UpperAcushnetRiver,1.0,41.681859,-70.918844,Buzzards Bay > Acushnet > Acushnet River
3,3,2,NewBedfordReservoir,1.0,41.681859,-70.918844,Buzzards Bay > Acushnet > Acushnet River
4,4,3,UpperAcushnetRiver,1.0,41.681859,-70.918844,Buzzards Bay > Acushnet > Acushnet River


In [4]:
# read the subwatershed polygons table
filename = "df_subs_tt_le5pct_select.pkl"
_ = pd.read_pickle(os.path.join(odr,filename))
print("search:",re.search("NA",r"^NA$"))
_.Travel_Tim = _.Travel_Tim.str.replace(pat="^NA$",repl="LT10",regex=True)
_ = _.loc[:,['SUBW_NAME','ele5pct','Travel_Tim','EMBAY_NAME','SUBWATER_N','Shape_Length','Shape_Area']].groupby(['SUBW_NAME','ele5pct','Travel_Tim']).aggregate(sum)
_ = _.reset_index()
print(_.info())
display(_.head())
df_subs = _
del _ # clear temporary object from memory

search: <re.Match object; span=(1, 3), match='NA'>
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1701 entries, 0 to 1700
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   SUBW_NAME     1701 non-null   object 
 1   ele5pct       1701 non-null   object 
 2   Travel_Tim    1701 non-null   object 
 3   Shape_Length  1701 non-null   float64
 4   Shape_Area    1701 non-null   float64
dtypes: float64(2), object(3)
memory usage: 66.6+ KB
None


,SUBW_NAME,ele5pct,Travel_Tim,Shape_Length,Shape_Area
0,,GT5%,,259703.644180,1.266581e+04
1,,LE5%,,49690.437654,3.250508e+03
2,,GT5%,LT10,583595.402837,1.272832e+08
3,,LE5%,LT10,412892.940536,1.603517e+07
4,4Ponds,GT5%,LT10,14768.416819,1.404234e+06


In [5]:
# read soil table
filename = "df_subs_tt_le5_soil20_select.pkl"
_ = pd.read_pickle(os.path.join(odr,filename))
# tidy up columns
_.rename(columns={'SLOPE_1':'SLOPE_pct'},inplace=True)
_.Travel_Tim = _.Travel_Tim.str.replace(pat="^NA$",repl="LT10",regex=True)
print(_.info())
display(_.head())
df_soil = _
del _ # clear temporary object from memory

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50068 entries, 1 to 50068
Data columns (total 27 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   SUBW_NAME     50068 non-null  object 
 1   ele5pct       50068 non-null  object 
 2   Travel_Tim    50068 non-null  object 
 3   EMBAY_NAME    50068 non-null  object 
 4   SUBWATER_N    50068 non-null  object 
 5   Shape_Length  50068 non-null  float64
 6   Shape_Area    50068 non-null  float64
 7   COMPNAME      50068 non-null  object 
 8   SLOPE         50068 non-null  object 
 9   SLOPE_pct     45277 non-null  float64
 10  FRMLNDCLS     49920 non-null  object 
 11  HYDROLGRP     43885 non-null  object 
 12  HYDRCRATNG    48192 non-null  object 
 13  DRAINCLASS    43674 non-null  object 
 14  DEP2WATTBL    20066 non-null  float64
 15  ROADS         50068 non-null  object 
 16  SEPTANKAF     50068 non-null  object 
 17  SLOPE_pct     45277 non-null  float64
 18  FLOODING      45710 non-nu

,SUBW_NAME,ele5pct,Travel_Tim,EMBAY_NAME,SUBWATER_N,Shape_Length,Shape_Area,COMPNAME,SLOPE,SLOPE_pct,...,SLOPE_pct,FLOODING,PONDING,CORCONCRET,PHWATER,CLAY,KSAT,OM,SAND,NLEACHING
OBJECTID,,,,,,,,,,,,,,,,,,,,,
1,,GT5%,,,,1050.566777,2.523144e+01,,,0.0,...,0.0,,,,0.0,0.0,0.0,0.0,0.0,
2,AdamsvilleBrook,GT5%,LT10,,AdamsvilleBrook,26601.966291,7.007353e+06,,,0.0,...,0.0,,,,0.0,0.0,0.0,0.0,0.0,
3,,GT5%,LT10,,,3406.694044,4.878317e+05,,,0.0,...,0.0,,,,0.0,0.0,0.0,0.0,0.0,
4,AdamsvilleBrook,LE5%,LT10,,AdamsvilleBrook,12904.924604,6.354489e+05,,,0.0,...,0.0,,,,0.0,0.0,0.0,0.0,0.0,
5,,LE5%,LT10,,,449.855742,1.744473e+01,,,0.0,...,0.0,,,,0.0,0.0,0.0,0.0,0.0,


In [108]:
# read in lclu table
filename = "df_subs_tt_le5_lclu16_select.pkl"
_ = pd.read_pickle(os.path.join(odr,filename))
# tidy up columns
_.Travel_Tim = _.Travel_Tim.str.replace(pat="^NA$",repl="LT10",regex=True)
print(_.info())
display(_.head())
df_lclu = _
del _ # clear temporary object from memory

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2988508 entries, 1 to 2988508
Data columns (total 12 columns):
 #   Column        Dtype  
---  ------        -----  
 0   SUBW_NAME     object 
 1   ele5pct       object 
 2   Travel_Tim    object 
 3   EMBAY_NAME    object 
 4   SUBWATER_N    object 
 5   Shape_Length  float64
 6   Shape_Area    float64
 7   USE_CODE      object 
 8   USEGENCODE    int64  
 9   COVERCODE     int64  
 10  COVERNAME     object 
 11  USEGENNAME    object 
dtypes: float64(2), int64(2), object(8)
memory usage: 296.4+ MB
None


,SUBW_NAME,ele5pct,Travel_Tim,EMBAY_NAME,SUBWATER_N,Shape_Length,Shape_Area,USE_CODE,USEGENCODE,COVERCODE,COVERNAME,USEGENNAME
OBJECTID,,,,,,,,,,,,
1,NashaquitsaPond,GT5%,LT10,MenemshaSquibnocketPond,NashaquitsaPond,4.705809,-0.001061,,0,0,,
2,ChilmarkPondLower,GT5%,LT10,ChilmarkPond,ChilmarkPondLower,9.029535,1.722165,1010,11,2,Impervious,Residential - single family
3,ChilmarkPondLower,GT5%,LT10,ChilmarkPond,ChilmarkPondLower,9.479830,1.314552,000,55,5,Developed Open Space,Right-of-way
4,ChilmarkPondLower,GT5%,LT10,ChilmarkPond,ChilmarkPondLower,127.319707,189.128979,000,55,5,Developed Open Space,Right-of-way
5,ChilmarkPondLower,GT5%,LT10,ChilmarkPond,ChilmarkPondLower,79.404249,98.960433,000,55,5,Developed Open Space,Right-of-way


In [132]:
# read in cranberry table
filename = "df_subs_tt_le5_cran_select.pkl"
_ = pd.read_pickle(os.path.join(odr,filename))
# tidy up columns
_.Travel_Tim = _.Travel_Tim.str.replace(pat="^NA$",repl="LT10",regex=True)
_.CRANBERRY = _.CRANBERRY.str.replace(pat="^$",repl="0",regex=True)
_.ACTIVE = _.ACTIVE.str.replace(pat="^$",repl="0",regex=True)
# remove rows where data is missing
_ = _.replace(r'^\s*$', np.nan, regex=True)
_ = _[_['SUBWATER_N'].notna()]
print(_.info())
display(_.head())
df_cran = _
del _ # clear temporary object from memory

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2663 entries, 3 to 2699
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   SUBW_NAME     2663 non-null   object 
 1   ele5pct       2663 non-null   object 
 2   Travel_Tim    2663 non-null   object 
 3   EMBAY_NAME    1813 non-null   object 
 4   SUBWATER_N    2663 non-null   object 
 5   CropStatus    544 non-null    object 
 6   WMA_NO        544 non-null    object 
 7   BOG_NAME      409 non-null    object 
 8   COMMENT       217 non-null    object 
 9   OWNER_FIRS    131 non-null    object 
 10  OWNER_LAST    488 non-null    object 
 11  COMMENT       217 non-null    object 
 12  CRANBERRY     2663 non-null   object 
 13  ACTIVE        2663 non-null   object 
 14  Shape_Length  2663 non-null   float64
 15  Shape_Area    2663 non-null   float64
dtypes: float64(2), object(14)
memory usage: 353.7+ KB
None


,SUBW_NAME,ele5pct,Travel_Tim,EMBAY_NAME,SUBWATER_N,CropStatus,WMA_NO,BOG_NAME,COMMENT,OWNER_FIRS,OWNER_LAST,COMMENT,CRANBERRY,ACTIVE,Shape_Length,Shape_Area
OBJECTID,,,,,,,,,,,,,,,,
3,Estuary,GT5%,LT10,NaN,Estuary,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,12606.261244,6.263356e+04
4,AgawamReservoirS,GT5%,LT10,NaN,AgawamReservoirSLT10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,54939.433736,5.906366e+06
5,FearingPond,GT5%,GT10,NaN,FearingPondGT10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,2726.583090,1.960601e+05
6,FearingPond,GT5%,LT10,NaN,FearingPondLT10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,6358.899527,1.025434e+06
7,LittleLongPond,GT5%,GT10,NaN,LittleLongPondGT10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,5326.940075,6.460576e+05


In [133]:
#Currently the calculation of cranberry acreage in terminus areas is messed up. Percentages are wayyy off. Need to look into this. . 
#Some land use groups also have percentages greater than 100. 

# summarize cranberry attributes by group
group_vars = ['SUBW_NAME','ele5pct','Travel_Tim']
selected_vars = ['CRANBERRY','ACTIVE']
value = 'Shape_Area' #,'Shape_Length']
df = df_cran

# loop 
i=0
for var in selected_vars:
    print(i,var)
    # there are two ways to pivot
    """
    # this is the same as pivot table
    _ = df.groupby(group_vars+[var])[value].sum()
    _ = _.pivot(index=group_vars,columns=var,values=value)
    _.reset_index(inplace=True) # this flattens column groupings
    """
    # pivot table is a one liner
    _ = df.pivot_table(index=group_vars, columns=var, values=value, aggfunc=np.sum, fill_value=0)
    _.columns = [var+"_"+str(s).strip() for s in _.columns.tolist()]
    _.reset_index(inplace=True)
    #print(_.columns)
    #print(_)
    if i == 0:
        df_ = _
    else:
        df_ = df_.merge(_,how="outer",on=group_vars)
    i+=1


0 CRANBERRY
1 ACTIVE


In [134]:
df_.to_csv(os.path.join(odr,"df_cran_pivot.csv"),index=False)
print(df_.info())
df_.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1697 entries, 0 to 1696
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   SUBW_NAME    1697 non-null   object 
 1   ele5pct      1697 non-null   object 
 2   Travel_Tim   1697 non-null   object 
 3   CRANBERRY_0  1697 non-null   float64
 4   CRANBERRY_1  1697 non-null   float64
 5   ACTIVE_0     1697 non-null   float64
 6   ACTIVE_1     1697 non-null   float64
dtypes: float64(4), object(3)
memory usage: 106.1+ KB
None


,SUBW_NAME,ele5pct,Travel_Tim,CRANBERRY_0,CRANBERRY_1,ACTIVE_0,ACTIVE_1
0,4Ponds,GT5%,LT10,1.404234e+06,0.0,1.404234e+06,0.0
1,4Ponds,LE5%,LT10,4.821770e+05,0.0,4.821770e+05,0.0
2,AbnerPond,GT5%,LT10,3.355843e+05,0.0,3.355843e+05,0.0
3,AbnerPond,LE5%,LT10,5.101133e+04,0.0,5.101133e+04,0.0
4,AdamsvilleBrook,GT5%,LT10,1.408436e+07,0.0,1.408436e+07,0.0


In [135]:
# summarize soil attributes by group
group_vars = ['SUBW_NAME','ele5pct','Travel_Tim']
selected_vars = ["SLOPE","FRMLNDCLS",'HYDROLGRP','HYDRCRATNG','DRAINCLASS','DEP2WATTBL','CLAY','OM','NLEACHING']
value = 'Shape_Area' #,'Shape_Length']
df = df_soil

# loop 
i=0
for var in selected_vars:
    print(i,var)
    # there are two ways to pivot
    """
    # this is the same as pivot table
    _ = df.groupby(group_vars+[var])[value].sum()
    _ = _.pivot(index=group_vars,columns=var,values=value)
    _.reset_index(inplace=True) # this flattens column groupings
    """
    # pivot table is a one liner
    _ = df.pivot_table(index=group_vars, columns=var, values=value, aggfunc=np.sum, fill_value=0)
    _.columns = [var+"_"+str(s).strip() for s in _.columns.tolist()]
    _.reset_index(inplace=True)
    #print(_.columns)
    #print(_)
    if i == 0:
        df_ = _
    else:
        df_ = df_.merge(_,how="outer",on=group_vars)
    i+=1


0 SLOPE
1 FRMLNDCLS
2 HYDROLGRP
3 HYDRCRATNG
4 DRAINCLASS
5 DEP2WATTBL
6 CLAY
7 OM
8 NLEACHING


In [136]:
df_.to_csv(os.path.join(odr,"df_soil_pivot.csv"),index=False)
print(df_.info())
df_.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1701 entries, 0 to 1700
Columns: 151 entries, SUBW_NAME to NLEACHING_Not rated
dtypes: float64(148), object(3)
memory usage: 2.0+ MB
None


,SUBW_NAME,ele5pct,Travel_Tim,SLOPE_,SLOPE_0,SLOPE_A,SLOPE_B,SLOPE_C,SLOPE_D,SLOPE_E,...,OM_10.0,OM_11.0,OM_12.0,OM_14.0,OM_25.0,NLEACHING_,NLEACHING_High,NLEACHING_Low,NLEACHING_Moderate,NLEACHING_Not rated
0,,GT5%,,25.231443,1.627878e+03,2.313204e+03,4.710167e+03,2.693368e+03,7.203493e+02,575.464972,...,1.221979e+03,2.131102,4.469022e+01,0.000000,0.0,25.231443,8.960348e+03,6.156343e+02,1.389399e+03,1.675049e+03
1,,LE5%,,0.000000,2.030214e+03,6.220418e+02,2.282130e+02,2.518022e+02,8.129251e+01,36.925844,...,2.759147e+02,0.000000,2.376740e+01,84.142761,0.0,0.000000,6.452343e+02,3.139557e+01,5.137930e+02,2.060066e+03
2,,GT5%,LT10,487831.698053,9.549469e+06,4.001676e+07,6.235737e+07,1.263683e+07,2.123605e+06,111382.725726,...,5.567864e+07,0.000000,3.433427e+06,0.000000,0.0,487831.698053,4.501328e+07,3.353700e+07,3.869566e+07,9.549469e+06
3,,LE5%,LT10,17.444732,1.367138e+07,2.156084e+06,9.077120e+04,8.633103e+04,2.974321e+04,849.149464,...,2.186967e+06,0.000000,2.024397e+03,0.000000,0.0,17.444732,1.780503e+05,1.297998e+04,2.172748e+06,1.367138e+07
4,4Ponds,GT5%,LT10,0.000000,1.323824e+03,4.729414e+05,7.631648e+05,1.588068e+05,0.000000e+00,7997.302077,...,0.000000e+00,0.000000,6.058967e+03,0.000000,0.0,0.000000,1.381209e+06,1.364389e+04,8.056370e+03,1.324385e+03


In [137]:
# summarize lclu attributes by group
group_vars = ['SUBW_NAME','ele5pct','Travel_Tim']
selected_vars = ["COVERNAME","USEGENNAME"]
value = 'Shape_Area' #,'Shape_Length']
df = df_lclu

# loop 
i=0
for var in selected_vars:
    print(i,var)
    # there are two ways to pivot
    """
    # this is the same as pivot table
    _ = df.groupby(group_vars+[var])[value].sum()
    _ = _.pivot(index=group_vars,columns=var,values=value)
    _.reset_index(inplace=True) # this flattens column groupings
    """
    # pivot table is a one liner
    _ = df.pivot_table(index=group_vars, columns=var, values=value, aggfunc=np.sum, fill_value=0)
    _.columns = [var+"_"+str(s).strip() for s in _.columns.tolist()]
    _.reset_index(inplace=True)
    #print(_.columns)
    #print(_)
    if i == 0:
        df_ = _
    else:
        df_ = df_.merge(_,on=group_vars)
    i+=1

0 COVERNAME
1 USEGENNAME


In [138]:
df_.to_csv(os.path.join(odr,"df_lclu_pivot.csv"),index=False)
print(df_.info())
df_.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1701 entries, 0 to 1700
Data columns (total 40 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   SUBW_NAME                                    1701 non-null   object 
 1   ele5pct                                      1701 non-null   object 
 2   Travel_Tim                                   1701 non-null   object 
 3   COVERNAME_                                   1701 non-null   float64
 4   COVERNAME_Bare Land                          1701 non-null   float64
 5   COVERNAME_Cultivated                         1701 non-null   float64
 6   COVERNAME_Deciduous Forest                   1701 non-null   float64
 7   COVERNAME_Developed Open Space               1701 non-null   float64
 8   COVERNAME_Estaurine Aquatic bed              1701 non-null   float64
 9   COVERNAME_Estuarine Emergent Wetland         1701 non-null   float64
 10  

,SUBW_NAME,ele5pct,Travel_Tim,COVERNAME_,COVERNAME_Bare Land,COVERNAME_Cultivated,COVERNAME_Deciduous Forest,COVERNAME_Developed Open Space,COVERNAME_Estaurine Aquatic bed,COVERNAME_Estuarine Emergent Wetland,...,"USEGENNAME_Mixed use, primarily residential",USEGENNAME_Open land,USEGENNAME_Recreation,USEGENNAME_Residential - multi-family,USEGENNAME_Residential - other,USEGENNAME_Residential - single family,USEGENNAME_Right-of-way,USEGENNAME_Tax exempt,USEGENNAME_Unknown,USEGENNAME_Water
0,,GT5%,,90.000510,873.363828,1.507682e+02,2.749985e+03,1.320793e+03,0.000000,2.695322e+02,...,221.454155,2.506013e+03,9.441592e+01,6.981094e+02,1.966737e+02,3.694296e+03,1.280920e+03,1.661155e+03,7.342159e+02,9.308387e+01
1,,LE5%,,25.240621,189.239845,0.000000e+00,1.270608e+02,9.138130e+01,0.532356,4.054123e+02,...,8.500700,6.179313e+02,2.599578e+01,6.253323e+01,3.801540e-01,3.706389e+02,4.487700e+01,1.863578e+02,1.650039e+03,1.734079e+02
2,,GT5%,LT10,488657.290602,743987.573631,2.597334e+06,3.891523e+07,8.355964e+06,0.000000,1.917891e+06,...,434254.852355,2.685085e+07,1.536038e+06,3.724654e+06,1.444183e+07,3.242442e+07,4.854466e+06,2.363364e+07,1.691464e+06,4.101059e+06
3,,LE5%,LT10,17.452497,10012.492031,1.918741e+02,2.459069e+04,5.825278e+03,0.000000,1.720496e+06,...,0.000000,2.642615e+05,5.197401e+03,5.991782e+04,2.106305e+05,2.039627e+05,4.402987e+03,1.262621e+05,1.470654e+07,3.865931e+05
4,4Ponds,GT5%,LT10,0.000000,79484.445747,0.000000e+00,2.237928e+05,2.911542e+05,0.000000,0.000000e+00,...,0.000000,3.584378e+05,1.040077e+04,5.344300e+04,0.000000e+00,3.960968e+05,5.299724e+04,4.755446e+04,0.000000e+00,2.471963e+03


In [139]:
# merge the pivot tables
df_soil_pivot = pd.read_csv(os.path.join(odr,"df_soil_pivot.csv"))
df_lclu_pivot = pd.read_csv(os.path.join(odr,"df_lclu_pivot.csv"))
df_cran_pivot = pd.read_csv(os.path.join(odr,"df_cran_pivot.csv"))
_ = pd.merge(df_soil_pivot,df_lclu_pivot,on=group_vars,how='right')
_ = _.merge(df_cran_pivot,on=group_vars,how='right')
_.info()
df_merged_pivot = _
df_merged_pivot.to_csv(os.path.join(odr,'df_merged_pivot.csv'))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1697 entries, 0 to 1696
Columns: 192 entries, SUBW_NAME to ACTIVE_1
dtypes: float64(189), object(3)
memory usage: 2.5+ MB


In [140]:
print(df_points.shape)
print(df_subs.shape)
print(df_merged_pivot.info())

(252, 7)
(1701, 5)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1697 entries, 0 to 1696
Columns: 192 entries, SUBW_NAME to ACTIVE_1
dtypes: float64(189), object(3)
memory usage: 2.5+ MB
None


In [141]:
# merge pivot with monitoring point dataset
#df_subs_points = df_subs.merge(df_points,on="SUBW_NAME")
#df_subs_points.to_csv(os.path.join(odr,"df_subs_points.csv"))
df_points_merged_pivot = df_points.merge(df_merged_pivot,on=["SUBW_NAME"])
df_points_merged_pivot.to_csv(os.path.join(odr,"df_points_merged_pivot.csv"))
df_points_merged_pivot.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 617 entries, 0 to 616
Columns: 198 entries, Unnamed: 0 to ACTIVE_1
dtypes: float64(192), int64(2), object(4)
memory usage: 959.2+ KB


In [142]:
# check if all names are in merged pivot
ps1 = df_merged_pivot
ps2 = df_subs
# Using Bitwise NOT operator along
# with pandas.isin()
print("\nItems of ps1 not present in ps2:")
res = ps1[~ps1.SUBW_NAME.isin(ps2.SUBW_NAME)].SUBW_NAME.unique()
print(res)


Items of ps1 not present in ps2:
[]


In [144]:
df_points_merged_pivot.merge(df_subs,how='left').Region_MEP.nunique()

96

In [145]:
# merge pivot points with subs to get total shape area for each sub
_ = df_points_merged_pivot.merge(df_subs,how='left')
print(_.columns)
print(_.info())
len(_.SUBW_NAME.unique())

cols = ['FID', 'SUBW_NAME', 'PROPORTION', 'Lat', 'Lon',#'EMBAY_NAME','SUBWATER_N',
       'Region_MEP', 'ele5pct', 'Travel_Tim', 'Shape_Length', 'Shape_Area']
# reordering columns 
_select = _[cols]
_.drop(['Unnamed: 0']+cols,axis=1,inplace=True)
_ = _select.join(_)
df_points_merged_pivot_subs = _
df_points_merged_pivot_subs.to_csv(os.path.join(odr,"df_points_merged_pivot_subs.csv"))
len(df_points_merged_pivot_subs.Region_MEP.unique())
df_points_merged_pivot_subs.head()

Index(['Unnamed: 0', 'FID', 'SUBW_NAME', 'PROPORTION', 'Lat', 'Lon',
       'Region_MEP', 'ele5pct', 'Travel_Tim', 'SLOPE_',
       ...
       'USEGENNAME_Right-of-way', 'USEGENNAME_Tax exempt',
       'USEGENNAME_Unknown', 'USEGENNAME_Water', 'CRANBERRY_0', 'CRANBERRY_1',
       'ACTIVE_0', 'ACTIVE_1', 'Shape_Length', 'Shape_Area'],
      dtype='object', length=200)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 617 entries, 0 to 616
Columns: 200 entries, Unnamed: 0 to Shape_Area
dtypes: float64(194), int64(2), object(4)
memory usage: 968.9+ KB
None


,FID,SUBW_NAME,PROPORTION,Lat,Lon,Region_MEP,ele5pct,Travel_Tim,Shape_Length,Shape_Area,...,USEGENNAME_Residential - other,USEGENNAME_Residential - single family,USEGENNAME_Right-of-way,USEGENNAME_Tax exempt,USEGENNAME_Unknown,USEGENNAME_Water,CRANBERRY_0,CRANBERRY_1,ACTIVE_0,ACTIVE_1
0,1,UpperAcushnetRiver,1.0,41.681859,-70.918844,Buzzards Bay > Acushnet > Acushnet River,GT5%,LT10,71601.933609,2.447743e+07,...,3899.61639,8.554857e+06,1.272751e+06,1.251882e+06,73301.147419,0.000000,2.436150e+07,115927.382405,2.439786e+07,79565.584148
1,1,UpperAcushnetRiver,1.0,41.681859,-70.918844,Buzzards Bay > Acushnet > Acushnet River,LE5%,LT10,41435.127851,1.402624e+06,...,0.00000,4.245029e+05,1.830879e+04,1.714518e+05,4935.680495,60866.420139,1.391156e+06,11468.200304,1.391156e+06,11468.200304
2,2,UpperAcushnetRiver,1.0,41.681859,-70.918844,Buzzards Bay > Acushnet > Acushnet River,GT5%,LT10,71601.933609,2.447743e+07,...,3899.61639,8.554857e+06,1.272751e+06,1.251882e+06,73301.147419,0.000000,2.436150e+07,115927.382405,2.439786e+07,79565.584148
3,2,UpperAcushnetRiver,1.0,41.681859,-70.918844,Buzzards Bay > Acushnet > Acushnet River,LE5%,LT10,41435.127851,1.402624e+06,...,0.00000,4.245029e+05,1.830879e+04,1.714518e+05,4935.680495,60866.420139,1.391156e+06,11468.200304,1.391156e+06,11468.200304
4,3,UpperAcushnetRiver,1.0,41.681859,-70.918844,Buzzards Bay > Acushnet > Acushnet River,GT5%,LT10,71601.933609,2.447743e+07,...,3899.61639,8.554857e+06,1.272751e+06,1.251882e+06,73301.147419,0.000000,2.436150e+07,115927.382405,2.439786e+07,79565.584148


In [146]:
z = df_points_merged_pivot_subs
# modify land use categories sum across rows 
selected = ['HYDROLGRP_A/D','HYDROLGRP_B/D','HYDROLGRP_C/D','HYDROLGRP_D']
z['HSG_D'] = z[selected].sum(axis=1)
z['HYDRIC'] = z['HYDRCRATNG_Yes']+z['HYDRCRATNG_Unranked']
selected = ['COVERNAME_Bare Land',
            'COVERNAME_Deciduous Forest',
                                       'COVERNAME_Estaurine Aquatic bed',
                                       'COVERNAME_Estuarine Emergent Wetland',
                                       'COVERNAME_Estuarine Forested Wetland',
                                       'COVERNAME_Estuarine Scrub/Shrub Wetland',
                                       'COVERNAME_Evergreen Forest',
                                       'COVERNAME_Grassland',
                                       'COVERNAME_Palustrine Aquatic Bed',
                                       'COVERNAME_Palustrine Emergent Wetland',
                                       'COVERNAME_Palustrine Forested Wetland',
                                       'COVERNAME_Palustrine Scrub/Shrub Wetland',
                                       'COVERNAME_Pasture/Hay',
                                       'COVERNAME_Scrub/Shrub',
                                       'COVERNAME_Unconsolidated Shore',
                                       'COVERNAME_Water']
z['NaturalCover'] = z[selected].sum(axis=1)
selected = ['USEGENNAME_Residential - multi-family',     
'USEGENNAME_Residential - other',
'USEGENNAME_Residential - single family']
z['Use_Res'] = z[selected].sum(axis=1)
selected = ['USEGENNAME_Residential - multi-family',     
'USEGENNAME_Residential - other',
'USEGENNAME_Residential - single family',
'USEGENNAME_Commercial',
'USEGENNAME_Mixed use, other',
'USEGENNAME_Mixed use, primarily commercial',
'USEGENNAME_Mixed use, primarily residential']
z['Use_ResComMix'] = z[selected].sum(axis=1)
selected = [
'USEGENNAME_Mixed use, other',
'USEGENNAME_Mixed use, primarily commercial',
'USEGENNAME_Mixed use, primarily residential']
z['Use_Mix'] = z[selected].sum(axis=1)

df_points_merged_pivot_subs = z

In [123]:
# LAND USE IS CRANBERRY 
# LAND USE IS ACTIVE CRANBERRY
# LAND USE IS RETIRED CRANBERRY
# LAND USE IS CONSERVED CRANBERRY

In [147]:
# Normalize watershed areas by proportion draining to sub
x = df_points_merged_pivot_subs
exclude_features = ['FID', 'SUBW_NAME', 'PROPORTION', 'Lat', 'Lon',#'EMBAY_NAME','SUBWATER_N',
       'Region_MEP', 'ele5pct', 'Travel_Tim', 'Shape_Length', 'Shape_Area']
value = 'PROPORTION'
_ = x.loc[:, ~x.columns.isin(exclude_features+[value])]
_ = _.select_dtypes(exclude=['object'])
y = _.mul(x[value],axis=0)
_ = pd.merge(x[exclude_features],y,left_index=True, right_index=True)
z = _.drop(["PROPORTION","Shape_Length"],axis=1) #remove the proportion column from z
print(len(z.FID.unique()))
z.to_csv(os.path.join(odr,"df_points_merged_pivot_subs_normalized.csv"))
df_points_merged_pivot_subs_normalized = z
z.head()

100


,FID,SUBW_NAME,Lat,Lon,Region_MEP,ele5pct,Travel_Tim,Shape_Area,SLOPE_,SLOPE_0,...,CRANBERRY_0,CRANBERRY_1,ACTIVE_0,ACTIVE_1,HSG_D,HYDRIC,NaturalCover,Use_Res,Use_ResComMix,Use_Mix
0,1,UpperAcushnetRiver,41.681859,-70.918844,Buzzards Bay > Acushnet > Acushnet River,GT5%,LT10,2.447743e+07,0.0,1.641069e+06,...,2.436150e+07,115927.382405,2.439786e+07,79565.584148,1.216630e+07,1.071397e+07,1.792842e+07,1.033690e+07,1.341410e+07,2.592965e+06
1,1,UpperAcushnetRiver,41.681859,-70.918844,Buzzards Bay > Acushnet > Acushnet River,LE5%,LT10,1.402624e+06,0.0,1.514730e+05,...,1.391156e+06,11468.200304,1.391156e+06,11468.200304,9.575296e+05,1.058379e+06,1.266456e+06,6.196251e+05,7.752412e+05,1.553943e+05
2,2,UpperAcushnetRiver,41.681859,-70.918844,Buzzards Bay > Acushnet > Acushnet River,GT5%,LT10,2.447743e+07,0.0,1.641069e+06,...,2.436150e+07,115927.382405,2.439786e+07,79565.584148,1.216630e+07,1.071397e+07,1.792842e+07,1.033690e+07,1.341410e+07,2.592965e+06
3,2,UpperAcushnetRiver,41.681859,-70.918844,Buzzards Bay > Acushnet > Acushnet River,LE5%,LT10,1.402624e+06,0.0,1.514730e+05,...,1.391156e+06,11468.200304,1.391156e+06,11468.200304,9.575296e+05,1.058379e+06,1.266456e+06,6.196251e+05,7.752412e+05,1.553943e+05
4,3,UpperAcushnetRiver,41.681859,-70.918844,Buzzards Bay > Acushnet > Acushnet River,GT5%,LT10,2.447743e+07,0.0,1.641069e+06,...,2.436150e+07,115927.382405,2.439786e+07,79565.584148,1.216630e+07,1.071397e+07,1.792842e+07,1.033690e+07,1.341410e+07,2.592965e+06


In [148]:
# Aggregate data by group
#_ = _.select_dtypes(exclude=['object'])
group_cols = ["FID","Region_MEP","Lat","Lon"]
df = df_points_merged_pivot_subs_normalized
x = df.groupby(group_cols).aggregate(sum).reset_index()
#_.pivot_table(index=['FID'], aggfunc=np.sum, fill_value=0)
#z.to_csv(os.path.join(odr,"z.csv"))

# calculate percent cover normalize metrics by polygon area 
value = 'Shape_Area'
#print(x.columns)
#_.columns.difference()
#exclude_features = group_vars+[value]
_ = x.loc[:, ~x.columns.isin(group_cols+[value])]
#print(_.columns)
#_ = _.select_dtypes(exclude=['object'])
y = _.div(x[value],axis=0).mul(100)
y.rename(columns={'Shape_Length':'Shape_Perim_to_Area'},inplace=True)
# merge the data back together
z = pd.merge(x[group_cols],x[value],left_index=True,right_index=True).merge(y,left_index=True, right_index=True)
z.head()

,FID,Region_MEP,Lat,Lon,Shape_Area,SLOPE_,SLOPE_0,SLOPE_A,SLOPE_B,SLOPE_C,...,CRANBERRY_0,CRANBERRY_1,ACTIVE_0,ACTIVE_1,HSG_D,HYDRIC,NaturalCover,Use_Res,Use_ResComMix,Use_Mix
0,1,Buzzards Bay > Acushnet > Acushnet River,41.681859,-70.918844,4.510600e+07,0.000000,7.346414,40.006569,41.155998,8.742853,...,97.766735,2.233265,97.926025,2.073975,46.221457,42.762381,76.187402,41.569732,52.911886,9.138170
1,2,Buzzards Bay > Acushnet > Acushnet River,41.681859,-70.918844,4.510600e+07,0.000000,7.346414,40.006569,41.155998,8.742853,...,97.766735,2.233265,97.926025,2.073975,46.221457,42.762381,76.187402,41.569732,52.911886,9.138170
2,3,Buzzards Bay > Acushnet > Acushnet River,41.681859,-70.918844,4.510600e+07,0.000000,7.346414,40.006569,41.155998,8.742853,...,97.766735,2.233265,97.926025,2.073975,46.221457,42.762381,76.187402,41.569732,52.911886,9.138170
3,4,Buzzards Bay > Acushnet > Acushnet River,41.681859,-70.918844,4.510600e+07,0.000000,7.346414,40.006569,41.155998,8.742853,...,97.766735,2.233265,97.926025,2.073975,46.221457,42.762381,76.187402,41.569732,52.911886,9.138170
4,5,Buzzards Bay > Westport > Adamsville Brook,41.553741,-71.126612,1.489267e+07,51.319207,0.620756,21.203183,25.675601,0.913079,...,100.000000,0.000000,100.000000,0.000000,29.687438,22.107823,43.198096,25.127047,26.365425,0.022352


In [150]:
z.to_csv(os.path.join(odr,"df_monitoring_point_sub_attributes.csv"),index=False)
df_monitoring_point_sub_attributes = z
z.head()

,FID,Region_MEP,Lat,Lon,Shape_Area,SLOPE_,SLOPE_0,SLOPE_A,SLOPE_B,SLOPE_C,...,CRANBERRY_0,CRANBERRY_1,ACTIVE_0,ACTIVE_1,HSG_D,HYDRIC,NaturalCover,Use_Res,Use_ResComMix,Use_Mix
0,1,Buzzards Bay > Acushnet > Acushnet River,41.681859,-70.918844,4.510600e+07,0.000000,7.346414,40.006569,41.155998,8.742853,...,97.766735,2.233265,97.926025,2.073975,46.221457,42.762381,76.187402,41.569732,52.911886,9.138170
1,2,Buzzards Bay > Acushnet > Acushnet River,41.681859,-70.918844,4.510600e+07,0.000000,7.346414,40.006569,41.155998,8.742853,...,97.766735,2.233265,97.926025,2.073975,46.221457,42.762381,76.187402,41.569732,52.911886,9.138170
2,3,Buzzards Bay > Acushnet > Acushnet River,41.681859,-70.918844,4.510600e+07,0.000000,7.346414,40.006569,41.155998,8.742853,...,97.766735,2.233265,97.926025,2.073975,46.221457,42.762381,76.187402,41.569732,52.911886,9.138170
3,4,Buzzards Bay > Acushnet > Acushnet River,41.681859,-70.918844,4.510600e+07,0.000000,7.346414,40.006569,41.155998,8.742853,...,97.766735,2.233265,97.926025,2.073975,46.221457,42.762381,76.187402,41.569732,52.911886,9.138170
4,5,Buzzards Bay > Westport > Adamsville Brook,41.553741,-71.126612,1.489267e+07,51.319207,0.620756,21.203183,25.675601,0.913079,...,100.000000,0.000000,100.000000,0.000000,29.687438,22.107823,43.198096,25.127047,26.365425,0.022352


In [151]:
# Aggregate data by group 
df_ele5pct = fn_agg_sum_df_on_group(group_cols=["FID","Region_MEP","ele5pct","Lat","Lon"],
                                    df=df_points_merged_pivot_subs_normalized,
                                    func=sum)
df_ele5pct.head()

,FID,Region_MEP,ele5pct,Lat,Lon,Shape_Area,SLOPE_,SLOPE_0,SLOPE_A,SLOPE_B,...,CRANBERRY_0,CRANBERRY_1,ACTIVE_0,ACTIVE_1,HSG_D,HYDRIC,NaturalCover,Use_Res,Use_ResComMix,Use_Mix
0,1,Buzzards Bay > Acushnet > Acushnet River,GT5%,41.681859,-70.918844,4.190077e+07,0.0,2.306279e+06,1.624800e+07,1.830513e+07,...,4.097338e+07,927395.683692,4.104523e+07,855546.342469,1.913142e+07,1.661481e+07,3.137421e+07,1.801710e+07,2.277192e+07,3.761169e+06
1,1,Buzzards Bay > Acushnet > Acushnet River,LE5%,41.681859,-70.918844,3.205225e+06,0.0,1.007394e+06,1.797358e+06,2.586906e+05,...,3.125284e+06,79940.603522,3.125284e+06,79940.603522,1.717232e+06,2.673592e+06,2.990876e+06,7.333425e+05,1.094514e+06,3.606934e+05
2,2,Buzzards Bay > Acushnet > Acushnet River,GT5%,41.681859,-70.918844,4.190077e+07,0.0,2.306279e+06,1.624800e+07,1.830513e+07,...,4.097338e+07,927395.683692,4.104523e+07,855546.342469,1.913142e+07,1.661481e+07,3.137421e+07,1.801710e+07,2.277192e+07,3.761169e+06
3,2,Buzzards Bay > Acushnet > Acushnet River,LE5%,41.681859,-70.918844,3.205225e+06,0.0,1.007394e+06,1.797358e+06,2.586906e+05,...,3.125284e+06,79940.603522,3.125284e+06,79940.603522,1.717232e+06,2.673592e+06,2.990876e+06,7.333425e+05,1.094514e+06,3.606934e+05
4,3,Buzzards Bay > Acushnet > Acushnet River,GT5%,41.681859,-70.918844,4.190077e+07,0.0,2.306279e+06,1.624800e+07,1.830513e+07,...,4.097338e+07,927395.683692,4.104523e+07,855546.342469,1.913142e+07,1.661481e+07,3.137421e+07,1.801710e+07,2.277192e+07,3.761169e+06


In [152]:
# split data by elevation percentile group
df_le5 = df_ele5pct[df_ele5pct.ele5pct == "LE5%"]
df_gt5 = df_ele5pct[df_ele5pct.ele5pct != "LE5%"]

In [153]:
# calculate percent cover for elevation percentile dataframes
df_le5_pct = fn_calc_pct_cover_within_groups(x=df_le5,area_col='Shape_Area',group_cols = ["FID","Region_MEP","Lat","Lon"])
#_.to_csv(os.path.join(odr,"df_monitoring_point_sub_attributes.csv"),index=False)
df_gt5_pct = fn_calc_pct_cover_within_groups(x=df_gt5,area_col='Shape_Area',group_cols = ["FID","Region_MEP","Lat","Lon"])

In [154]:
# add prefixes to selected columns that summarize areas above and below elevation threshold
all_colnames = df_le5_pct.columns
selected_cols = all_colnames.drop(group_cols)
selected_cols
df_le5_pct_pref = fn_add_prefix_suffix_to_selected_cols(df_gt5_pct,col_names=selected_cols,prefix='GT5')
df_gt5_pct_pref = fn_add_prefix_suffix_to_selected_cols(df_le5_pct,col_names=selected_cols,prefix='LE5')
df_gt5_pct_pref.head()

,FID,Region_MEP,Lat,Lon,LE5_Shape_Area,LE5_SLOPE_,LE5_SLOPE_0,LE5_SLOPE_A,LE5_SLOPE_B,LE5_SLOPE_C,...,LE5_CRANBERRY_0,LE5_CRANBERRY_1,LE5_ACTIVE_0,LE5_ACTIVE_1,LE5_HSG_D,LE5_HYDRIC,LE5_NaturalCover,LE5_Use_Res,LE5_Use_ResComMix,LE5_Use_Mix
1,1,Buzzards Bay > Acushnet > Acushnet River,41.681859,-70.918844,3.205225e+06,0.000000,31.429749,56.075867,8.070903,2.303626,...,97.505928,2.494072,97.505928,2.494072,53.576021,83.413571,93.312538,22.879596,34.147798,11.253295
3,2,Buzzards Bay > Acushnet > Acushnet River,41.681859,-70.918844,3.205225e+06,0.000000,31.429749,56.075867,8.070903,2.303626,...,97.505928,2.494072,97.505928,2.494072,53.576021,83.413571,93.312538,22.879596,34.147798,11.253295
5,3,Buzzards Bay > Acushnet > Acushnet River,41.681859,-70.918844,3.205225e+06,0.000000,31.429749,56.075867,8.070903,2.303626,...,97.505928,2.494072,97.505928,2.494072,53.576021,83.413571,93.312538,22.879596,34.147798,11.253295
7,4,Buzzards Bay > Acushnet > Acushnet River,41.681859,-70.918844,3.205225e+06,0.000000,31.429749,56.075867,8.070903,2.303626,...,97.505928,2.494072,97.505928,2.494072,53.576021,83.413571,93.312538,22.879596,34.147798,11.253295
9,5,Buzzards Bay > Westport > Adamsville Brook,41.553741,-71.126612,8.083125e+05,78.614256,11.417800,3.955511,6.012432,0.000000,...,100.000000,0.000000,100.000000,0.000000,1.712729,13.130529,13.727705,6.391487,16.517127,0.000000


In [155]:
# merge all pct cover datasets and save
df_monitoring_point_sub_attributes_terminus = df_monitoring_point_sub_attributes.merge(df_le5_pct_pref,how='outer').merge(df_gt5_pct_pref,how='outer')
# save final dataset to output directory
df_monitoring_point_sub_attributes_terminus.to_csv(os.path.join(odr,'df_monitoring_point_sub_attributes_terminus.csv'),index=False)

In [ ]:
## Unused code snippets